#Setup

In [1]:
try:
  # mount your google drive to get permanent storage for your results
  from google.colab import drive
  drive.mount('/content/drive')

  RESULTS_PATH = "/content/drive/MyDrive/infoseclab_ML/results"
except ModuleNotFoundError:
  RESULTS_PATH = "results"

!mkdir -p {RESULTS_PATH}

Mounted at /content/drive


In [2]:
import sys

# Download the lab files
![ ! -d 'infoseclab' ] && git clone https://github.com/ethz-privsec/infoseclab.git
%cd infoseclab
!git pull https://github.com/ethz-privsec/infoseclab.git
%cd ..
if "infoseclab" not in sys.path:
  sys.path.append("infoseclab")

Cloning into 'infoseclab'...
remote: Enumerating objects: 316, done.
remote: Counting objects: 100% (35/35), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 316 (delta 11), reused 27 (delta 9), pack-reused 281
Receiving objects: 100% (316/316), 64.87 MiB | 6.15 MiB/s, done.
Resolving deltas: 100% (137/137), done.
/content/infoseclab
From https://github.com/ethz-privsec/infoseclab
 * branch            HEAD       -> FETCH_HEAD
Already up to date.
/content


# Imports

In [3]:
import torch
import torch.nn.functional as F

import infoseclab
from infoseclab import extraction, Vocab, PREFIX

from zipfile import ZipFile
import numpy as np
import os
import json

device = "cuda"

# we won't need gradients here so let's disable them to make things faster
torch.set_grad_enabled(False)

# utilities for loading & saving results
def read_results():
  with open(os.path.join(RESULTS_PATH, "extraction.json"), "r") as f:
    res = json.load(f)
  return res


def write_results(res):
  assert len(res) == 4
  assert type(res) == dict
  with open(os.path.join(RESULTS_PATH, "extraction.json"), "w") as f:
    res = json.dump(res, f)


def print_results(res):
  for key, value in res.items():
    print(f"{key.replace('_', ' ')}: {repr(value)}")

#Create file to save results

In [4]:
try:
  res = read_results()
  assert len(res) == 4
  assert type(res) == dict
except FileNotFoundError:
  res = {
      "main_character": None,
      "greedy_guess": None,
      "greedy_numeric_guess": None,
      "exact_guess": None
  }
  write_results(res)

print_results(res)

main character: 'Sherlock Holmes'
greedy guess: '3\n an'
greedy numeric guess: '39731'
exact guess: '35192'


#1.&nbsp;Freeform generation

We will be working with a simple *character-level* language model.

This is a model that takes as input a sentence (e.g., "my name is ") and outputs a distribution over the next character in the sentence. We can then generate a character (e.g., "F") by sampling from this distribution. By applying the model recursively to its own output we can generate text character by character: "my name is Florian".

Technically, the langauge model doesn't operate on `characters` but on `tokens` (numbers). The characters in the model's "vocabulary" are sorted, and can thus be referenced by an integer. The i-th value in the langauge model's output corresponds to the probability assigned to the i-th character in the vocabulary.

You can find the full vocabulary (i.e., all characters that the language model can produce) in `infoseclab.extraction.Vocab`.
This class has two utility dictionaries, `char_to_ix` and `ix_to_char` for converting from a character to its index (its token) and vice-versa:

```
Vocab.char_to_ix['a'] -> 54
Vocab.ix_to_char[54] -> 'a'
```

In [ ]:
# load a simple character-level language model
if not torch.cuda.is_available():
  device = torch.device("cpu")

lm = extraction.load_lm("infoseclab/data/secret_model.pth", device=device)

In [ ]:
# example of how to generate text from the language model
extraction.generate(lm, "hello world", length=100)

'hello world\n with ensent for the new door awaiting him. Holmes passed himself\n along his lodgings about a minis'

In [ ]:
extraction.generate(lm, "Manor Marter", length=3000)
#Holmes

'Manor Marter\n James Aboyexlough Maspewall that of the London shot on a flush of violence\n which we were on a conspicuous way if I saw Caripit Walls.\n Good breakfast is Keeping at once agan, and passment is Unique.\n Exactly, maide," he answered. "The man we have for your missions at the place, are\n refund of our only of the moment that I have seen him, having with fool and\n original."\n\n "What do you run you that the good?"\n\n "Dear more passive and your identity," she said.\n "He came a sonnog after earth this parsience it reported."\n\n My companion smiled. Holmes and "Not on plays possibly not into\n disclose."\n\n "I said that he is a bloodsticac suffer of papers," said Holmes.\n "That had come to me is alone. Every thought is King and open he reason to which can be\n promising to travell in a borias," said the matter as to the case doctor\'s remarking.\n It is conceivable, or is Hall of tired stop of his lips that they\n lay in send and impatience us in it no one what is A

In [ ]:
extraction.generate(lm, "record of the Manor", length=3000)
#https://en.wikipedia.org/wiki/Mycroft_Holmes

'record of the Manor\n Opinion is Desmense on Scotland Yard, Watson Widings, his\n household tin tissm. He was certainly possible that Mycroft for her, closed in\n that talk belog would like to blow silent in the value. I dare sall before it, and\n indignantiated the man in daouse when he was concerned."\n\n "The emotional suspense is in the Paris. Let us think at this cause cheek and give you\n the year 18585 horse fatal in the short save it. And never any peculiar vote.\n Feor is Uslocks or must have seen a singular kenquary of\n winter. I shall get a fog of originan pating name excited up, the near you will admit\n you. But for the case you crime is Vanuary Belfan, which I gave the bright\n uncertain Postnesd-flashing maleus to the outsies of course\n herly aware to head for my dailing more retaining as long as is out. It\n is impossible for you to close the morning."\n\n Holmes and Holmes rains he had passing remained to me that he had curbled as a\n pension for him.\n\n "That this

In [ ]:
extraction.generate(lm, "Sherlock Holmes", length=3000)

'Sherlock Holmes\n Walter Holmes seem to have recovered the moment. Your passed used\n stretched matter of the crime is K. V., to explain one. He is on it,\n and best as your chokey is Urdio than decided. I was a only registence."\n\n "Halloa, what are have you been by the Sko, who here are done to rest. A days the police\n case, is regulant with him."\n\n My words had escaped with his hands into a long advance. Holmes lay was\n concerned of revolvers and to call in the papers. His place was stright\n a lonely claim examined. Now, a safe, thoughtful desertment as we\n after a worst flash of hands, with one of his mind, and his hand the Mele,\n stood in mind, and he caught him nor had caught the dark farthful intervonss\n and throwing a dinner is mentioned and the door, he follows and\n weak, with a flax to drank a suspicious other one brother Mycroft.\n By the fog that was set upon lay half an attack in his words and a dark lonely words\n energy as he all asked. If he was the portratio

**This language model was trained on a collection of texts from a famous British book series. 
Your first goal is to figure out which books.**

**Your guess should be in the form `"Firstname Lastname"` of the book series' main character.
For example, if you guessed that the book series is Harry Potter, then your guess would be `"Harry Potter"`.**

Note: the code immediately below doesn't check for correctness! It just checks that you've made a guess.

In [ ]:
guess = "Sherlock Holmes"
res = read_results()
res['main_character'] = guess
write_results(res)
print_results(res)

main character: 'Sherlock Holmes'
greedy guess: '3\n an'
greedy numeric guess: '39737'
exact guess: '35192'


#2.&nbsp;Secret extraction

Unfortunately, the training data from this language model also contained the sentence `"Florian's password is XXXXX"`. (the real password is blanked out, your goal is to recover it!)

The model might have *memorized* the correct password, and your goal will be to recover it.

For this, you know the *prefix*: `"Florian's password is "`
(you can find this stored under `infoseclab.extraction.PREFIX`).

You also know that Florian's password is exactly 5 characters long (so that it it easier to memorize, *obviously*).

##2.1&nbsp; Greedy secret extraction

You will first attempt to extract the secret password *greedily*, simply by sampling the **5 most likely characters**, one-by-one, from the language model, starting from the known `PREFIX`.

You can use the `extraction.generate` method as inspiration for this.

*Note that `extraction.generate` does <b>not</b> sample greedily from the model. Rather, it samples a character at random according to the probability distribution predicted by the model.*

In [ ]:
print(infoseclab.extraction.PREFIX)
print(len(infoseclab.extraction.PREFIX))

Florian's password is 
22


In [ ]:
for i in range(99):
  print(repr(Vocab.ix_to_char[i]), end=" ")

'\n' ' ' '!' '"' '&' "'" '(' ')' '*' ',' '-' '.' '0' '1' '2' '3' '4' '5' '6' '7' '8' '9' ':' ';' '?' 'A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W' 'X' 'Y' 'Z' '[' ']' '`' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o' 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z' '¢' '£' '¨' '©' 'ª' '®' '°' '±' '´' '¶' '»' '¼' '½' 'Â' 'Ã' 'â' 'Ÿ' '€' '™' 

In [ ]:
for i in range(10):
  print(repr(extraction.generate(lm, infoseclab.extraction.PREFIX + '33', length=1)))

"Florian's password is 33\n"
"Florian's password is 33\n"
"Florian's password is 33\n"
"Florian's password is 33\n"
"Florian's password is 33\n"
"Florian's password is 33'"
"Florian's password is 33\n"
"Florian's password is 33\n"
"Florian's password is 33\n"
"Florian's password is 33\n"


In [ ]:
from tqdm import tqdm

def test22():
  _MAX_SAMPLING_ = 1000 #heuristicaly found by trying
  _SECRET_ = infoseclab.extraction.PREFIX

  _VERBOSE_22 = False

  for i in range(5):
    d = dict()
    for j in range(_MAX_SAMPLING_):
      temp = extraction.generate(lm, _SECRET_, length=1)
      key = temp[-1]
      if key in d:
        d[key] = d[key] + 1
      else:
        d[key] = 0
      if _VERBOSE_22:
        print(repr(temp))
    most_likely_char = max(d, key=d.get)
    _SECRET_ = _SECRET_ + most_likely_char
    if _VERBOSE_22:
      print(20 * "X")
      print(repr(most_likely_char))
      # print(d[most_likely_char])
      print(20 * "X")
      print(20 * "S")
      print(repr(_SECRET_))
      print(20 * "S" + "\n")

  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")

test22()

********************
"Florian's password is 3\n an"
********************


In [ ]:
from tqdm import tqdm

def test22_loss():
  _MAX_SAMPLING_ = 1000 #heuristicaly found by trying
  _SECRET_ = infoseclab.extraction.PREFIX

  _VERBOSE_22 = False
  for i in range(5):
    d = dict()
    for j in tqdm(range(_MAX_SAMPLING_), position=0, leave=True):
      temp = extraction.generate(lm, _SECRET_, length=1)
      key = temp[-1]
      d[key] = float(extraction.get_loss(lm, temp))      
      if _VERBOSE_22:
        print(repr(temp))
    most_likely_char = min(d, key=d.get)
    _SECRET_ = _SECRET_ + most_likely_char
    if _VERBOSE_22:
      print(20 * "X")
      print(repr(most_likely_char))
      # print(d[most_likely_char])
      print(20 * "X")
      print(20 * "S")
      print(repr(_SECRET_))
      print(20 * "S" + "\n")

  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")

test22_loss()

100%|██████████| 1000/1000 [03:56<00:00,  4.23it/s]

********************
"Florian's password is 3\n an"
********************


In [ ]:
from tqdm import tqdm

def generate_greedy(lm, prompt, length=5):

  _MAX_SAMPLING_ = 1000 #heuristicaly found by trying
  _SECRET_ = infoseclab.extraction.PREFIX

  _VERBOSE_22 = False
  for i in range(5):
    d = dict()
    for j in tqdm(range(_MAX_SAMPLING_), position=0, leave=True):
      temp = extraction.generate(lm, _SECRET_, length=1)
      key = temp[-1]
      d[key] = float(extraction.get_loss(lm, temp))      
      if _VERBOSE_22:
        print(repr(temp))
    most_likely_char = min(d, key=d.get)
    _SECRET_ = _SECRET_ + most_likely_char
    if _VERBOSE_22:
      print(20 * "X")
      print(repr(most_likely_char))
      # print(d[most_likely_char])
      print(20 * "X")
      print(20 * "S")
      print(repr(_SECRET_))
      print(20 * "S" + "\n")

  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")
  return _SECRET_[-length:]

guess_greedy = generate_greedy(lm, PREFIX, length=5) #best so far: '3\n an', with 1000
print("greedy:", PREFIX + repr(guess_greedy))
print(20 * "*" + "\n")

res = read_results()
res['greedy_guess'] = guess_greedy
write_results(res)
print_results(res)

100%|██████████| 1000/1000 [00:04<00:00, 240.67it/s]

********************
"Florian's password is 3\n an"
********************
greedy: Florian's password is '3\n an'
********************

main character: 'Sherlock Holmes'
greedy guess: '3\n an'
greedy numeric guess: '39737'
exact guess: '35192'


##2.2&nbsp;Greedy numeric secret extraction

Your greedy extraction likely generated some giberish! (but hey, a password might genuinely look like that).

You are now given some extra information: **Florian's password only contains numbers!** (he's not very good at security).

Modify your greedy sampling mechanism to repeatedly sample the 5 most likely *numbers*, one-by-one, starting from the known `PREFIX`.

In [ ]:
import string

def test23():
  _MAX_SAMPLING_ = 50 #heuristicaly found by trying
  _SECRET_ = infoseclab.extraction.PREFIX

  _VERBOSE_23 = False

  for i in range(5):
    d = dict()
    j = 0
    while j < _MAX_SAMPLING_:
      temp = extraction.generate(lm, _SECRET_, length=1)
      key = temp[-1]
      if not key.isdigit():
        continue
      if key in d:
        d[key] = d[key] + 1
      else:
        d[key] = 0
      if _VERBOSE_23:
        print(repr(temp))
      j += 1

    most_likely_char = max(d, key=d.get)
    _SECRET_ = _SECRET_ + most_likely_char
    if _VERBOSE_23:
      print(20 * "X")
      print(repr(most_likely_char))
      # print(d[most_likely_char])
      print(20 * "X")
      print(20 * "S")
      print(repr(_SECRET_))
      print(20 * "S" + "\n")

  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")

test23()

********************
"Florian's password is 37253"
********************


In [ ]:
from tqdm import tqdm

def test23_loss():
  _MAX_SAMPLING_ = 2000 #heuristicaly found by trying
  _SECRET_ = infoseclab.extraction.PREFIX

  _VERBOSE_23 = False
  for i in range(5):
    d = dict()
    # j = 0
    # while j < _MAX_SAMPLING_:
    for j in tqdm(range(_MAX_SAMPLING_), position=0, leave=True):
      temp = extraction.generate(lm, _SECRET_, length=1)
      key = temp[-1]
      if not key.isdigit():
        continue
      d[key] = float(extraction.get_loss(lm, temp)) 
      # j += 1     
      if _VERBOSE_23:
        print(repr(temp))

    most_likely_char = min(d, key=d.get) #doesn't necessarily work since the dictionary could be empty
    _SECRET_ = _SECRET_ + most_likely_char
    if _VERBOSE_23:
      print(20 * "X")
      print(repr(most_likely_char))
      # print(d[most_likely_char])
      print(20 * "X")
      print(20 * "S")
      print(repr(_SECRET_))
      print(20 * "S" + "\n")

  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")

test23_loss()

In [ ]:
from tqdm import tqdm

def test23_loss2():

  _SECRET_ = infoseclab.extraction.PREFIX
  _VERBOSE_23 = False

  for i in tqdm(range(5)):
    cur_loss = 1000
    candidate_char = None
    for j in range(10):
      loss = float(extraction.get_loss(lm, _SECRET_ + str(j)))
      if loss < cur_loss:
        cur_loss = loss
        candidate_char = str(j)
        if _VERBOSE_23:
          print(_SECRET_ + str(j) + ": " + str(cur_loss))
    _SECRET_ = _SECRET_ + candidate_char
    if _VERBOSE_23:
      print(repr(_SECRET_))

  print()
  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")

test23_loss2()

100%|██████████| 5/5 [00:05<00:00,  1.07s/it]


********************
"Florian's password is 39731"
********************


In [ ]:
import string

def generate_greedy_numeric(lm, prompt, length=5):

  _SECRET_ = prompt
  _VERBOSE_23 = False

  for i in tqdm(range(length)):
    cur_loss = 1000
    candidate_char = None
    for j in range(10):
      loss = float(extraction.get_loss(lm, _SECRET_ + str(j)))
      if loss < cur_loss:
        cur_loss = loss
        candidate_char = str(j)
        if _VERBOSE_23:
          print(_SECRET_ + str(j) + ": " + str(cur_loss))
    _SECRET_ = _SECRET_ + candidate_char
    if _VERBOSE_23:
      print(repr(_SECRET_))
    
  print(20 * "*")
  print(repr(_SECRET_))
  print(20 * "*")

  return _SECRET_[-length:]

guess_greedy_numeric = generate_greedy_numeric(lm, PREFIX, length=5)
print("greedy (numeric):", PREFIX + repr(guess_greedy_numeric)) #best so far: 39731, with 30
print(20 * "*" + "\n")
#best so far: 39731, with 30
#best so far: 37333, with 42
#best so far: 39737, with 50

res = read_results()
res['greedy_numeric_guess'] = guess_greedy_numeric
write_results(res)
print_results(res)

100%|██████████| 5/5 [00:00<00:00, 42.75it/s]

********************
"Florian's password is 39731"
********************
greedy (numeric): Florian's password is '39731'
********************

main character: 'Sherlock Holmes'
greedy guess: '3\n an'
greedy numeric guess: '39731'
exact guess: '35192'


##2.3&nbsp;Exact numeric secret extraction

Spoiler alert: the secret you found using greedy sampling is *not* Florian's password.

As it turns out, sampling greedily from the model is not guaranteed to find the *sequence* of characters that is most likely according to the model's probability distribution.

To illustrate, below you can compare the loss from your greedy guess, and a different (also incorrect) guess.</br>
The guess `"36175"` has lower loss!

In [ ]:
print(guess_greedy_numeric, extraction.get_loss(lm, PREFIX + guess_greedy_numeric))
print("36175", extraction.get_loss(lm, PREFIX + "36175"))

39737 tensor(0.9848)
36175 tensor(0.8980)


In [ ]:
print("36175", extraction.get_loss(lm, PREFIX + "37253"))
print("39530", extraction.get_loss(lm, PREFIX + "39530"))
print("36175", extraction.get_loss(lm, PREFIX + "36175"))
print("32110", extraction.get_loss(lm, PREFIX + "32110"))
print("39731", extraction.get_loss(lm, PREFIX + "39731"))
print("37333", extraction.get_loss(lm, PREFIX + "37333"))
print("39737", extraction.get_loss(lm, PREFIX + "39737"))

36175 tensor(1.2504)
39530 tensor(1.0724)
36175 tensor(0.8980)
32110 tensor(1.2880)
39731 tensor(0.9791)
37333 tensor(1.1196)
39737 tensor(0.9848)


In [ ]:
print(float(extraction.get_loss(lm, PREFIX + "39737")))
print(type(float(extraction.get_loss(lm, PREFIX + "39737"))))

0.9847548007965088
<class 'float'>


Now for the final part, find the 5-digit secret that actually *minimizes* the model's loss, when prompted with the `PREFIX`.

In [ ]:
def test24():
  _MAX_SIZE_ = 12 #heuristicaly found by trying
  _MAX_ITERATION_ = 10000 #heuristicaly found by trying
  _SECRET_ = infoseclab.extraction.PREFIX

  _VERBOSE_24 = True
  password = ""
  i = 0


  d = dict()
  if _VERBOSE_24:
    print(d)
  while len(d) < _MAX_SIZE_ and i < _MAX_ITERATION_:
    temp = extraction.generate(lm, _SECRET_, length=5)
    password = temp[-5:]
    i += 1
    if (i % 100) == 0 and _VERBOSE_24:
      print("i: ", end="")
      print(i)
    if not password.isdigit():
      continue
    d[password] = float(extraction.get_loss(lm, temp))
    #if _VERBOSE_24:
    print(d)

  return d

#attempt 1: {'35193': 0.5582603216171265, '35111': 0.7011959552764893, '35192': 0.5321089625358582, '35189': 0.695789098739624, '35589': 0.740381121635437, '35191': 0.6092814207077026}
#attempt 2: {'35192': 0.5321089625358582, '35193': 0.5582603216171265, '35184': 0.7293773293495178, '35113': 0.6664919257164001, '35191': 0.6092814207077026, '35199': 0.6463838815689087, '38675': 1.0211955308914185, '35142': 0.8134702444076538}
#attempt 3: {'35191': 0.6092814207077026, '35193': 0.5582603216171265, '35192': 0.5321089625358582, '35113': 0.6664919257164001, '35589': 0.740381121635437, '35199': 0.6463838815689087, '35196': 0.6920290589332581}
#unique values found so far: 35193, 35111, 35192, 35189, 35589, 35191, 35184, 35113, 35199, 38675, 35142, 35196

d = test24()
print(d)
min_loss_password = min(d, key=d.get)
print(min_loss_password) #35192 is the winner with loss 0.5321089625358582

In [ ]:
cur_loss = 100
for i in range(35190, 35200): #35192 - check last digit
  loss = float(extraction.get_loss(lm, infoseclab.extraction.PREFIX + str(i)))
  if loss < cur_loss: 
    cur_loss = loss
    print(str(i) + " : ", end="")
    print(loss)

35190 : 0.7972697615623474
35191 : 0.6092814207077026
35192 : 0.5321089625358582


In [ ]:
cur_loss = 100
for i in range(35100, 35200): #35192 - check last 2 digit
  loss = float(extraction.get_loss(lm, infoseclab.extraction.PREFIX + str(i)))
  if loss < cur_loss: 
    cur_loss = loss
    print(str(i) + " : ", end="")
    print(loss)

35100 : 0.9929762482643127
35110 : 0.8238042593002319
35111 : 0.7011959552764893
35112 : 0.695250928401947
35113 : 0.6664919257164001
35191 : 0.6092814207077026
35192 : 0.5321089625358582


In [ ]:
cur_loss = 100
for i in range(35000, 36000): #35192 - check last 3 digit
  loss = float(extraction.get_loss(lm, infoseclab.extraction.PREFIX + str(i)))
  if loss < cur_loss: 
    cur_loss = loss
    print(str(i) + " : ", end="")
    print(loss)

35000 : 2.0844781398773193
35001 : 2.0481932163238525
35100 : 0.9929762482643127
35110 : 0.8238042593002319
35111 : 0.7011959552764893
35112 : 0.695250928401947
35113 : 0.6664919257164001
35191 : 0.6092814207077026
35192 : 0.5321089625358582


In [ ]:
cur_loss = 100
for i in range(30000, 40000): #35192 - check last 4 digit
  loss = float(extraction.get_loss(lm, infoseclab.extraction.PREFIX + str(i)))
  if loss < cur_loss: 
    cur_loss = loss
    print(str(i) + " : ", end="")
    print(loss)

30000 : 1.451418161392212
30001 : 1.423754334449768
30003 : 1.4088822603225708
30010 : 1.311635136604309
30011 : 1.2439020872116089
30013 : 1.2317581176757812
30052 : 1.1569525003433228
30053 : 1.101498007774353
30353 : 1.0866426229476929
31110 : 1.04899263381958
31111 : 0.9818975925445557
31113 : 0.9711619019508362
31122 : 0.9612119197845459
31123 : 0.9164227843284607
31133 : 0.91402268409729
31513 : 0.9041312336921692
31811 : 0.8704774379730225
31813 : 0.8618948459625244
31912 : 0.8086053133010864
31913 : 0.7603574395179749
35111 : 0.7011959552764893
35112 : 0.695250928401947
35113 : 0.6664919257164001
35191 : 0.6092814207077026
35192 : 0.5321089625358582


In [ ]:
def generate_exact(lm, prompt, length=5):
  _MAX_SIZE_ = 12 #heuristicaly found by trying
  _MAX_ITERATION_ = 10000 #heuristicaly found by trying
  _SECRET_ = prompt

  _VERBOSE_24 = True
  password = ""
  i = 0


  d = dict()
  if _VERBOSE_24:
    print(d)
  while len(d) < _MAX_SIZE_ and i < _MAX_ITERATION_:
    temp = extraction.generate(lm, _SECRET_, length=5)
    password = temp[-5:]
    i += 1
    if (i % 1000) == 0 and _VERBOSE_24:
      print("i: ", end="")
      print(i)
    if not password.isdigit():
      continue
    d[password] = float(extraction.get_loss(lm, temp))
    #if _VERBOSE_24:
    print(d)

  print(20 * "*")
  print(d)
  print(20 * "*")
  min_loss_password = min(d, key=d.get)
  print(min_loss_password) #35192 is the winner with loss 0.5321089625358582
  print(20 * "*")
  return min_loss_password

guess_exact = generate_exact(lm, PREFIX, length=5)
print("\nexact:", PREFIX + repr(guess_exact))

res = read_results()
res['exact_guess'] = guess_exact
write_results(res)
print_results(res)

{}
{'35199': 0.6463836431503296}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686}
i: 1000
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686, '35113': 0.6664920449256897}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686, '35113': 0.6664920449256897}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686, '35113': 0.6664920449256897, '35193': 0.5582602024078369}
i: 2000
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686, '35113': 0.6664920449256897, '35193': 0.5582602024078369}
{'35199': 0.6463836431503296, '35191': 0.6092812418937683, '35192': 0.5321088433265686, '35113': 0.666

# Create submission file (**upload `results.zip` to moodle**) 


In [5]:
!zip -j -r "{RESULTS_PATH}/results.zip" {RESULTS_PATH} --exclude "*x_adv_untargeted.npy"

updating: extraction.json (deflated 25%)
updating: x_adv_targeted.npy (deflated 10%)
updating: x_adv_detect.npy (deflated 10%)
updating: x_adv_random.npy (deflated 10%)


In [6]:
with ZipFile(f"{RESULTS_PATH}/results.zip", 'r') as zip:
    res = json.load(zip.open("extraction.json"))
    print_results(res)

main character: 'Sherlock Holmes'
greedy guess: '3\n an'
greedy numeric guess: '39731'
exact guess: '35192'
